# Robyn: Marketing Mix Modeling Application

This notebook demonstrates the usage of Robyn, a Marketing Mix Modeling (MMM) application. 
We'll go through the main steps of performing robyn_inputs and robyn_engineering.



## 1. Import Required Libraries. Define Paths.

First, be sure to setup your virtual environment. Be sure to switch over to your new environment in this notebook. 

-```cd {root_folder}```

-```python3 -m yourvenv```

-```source yourvenv/bin/activate```

-```cd Robyn/python```

-```pip install -r requirements.txt```


Then import the necessary libraries. Make sure to define your paths below.



In [ ]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [ ]:
import os
import pandas as pd
from typing import Dict
from robyn.data.entities.mmmdata import MMMData
from robyn.data.entities.enums import AdstockType
from robyn.data.entities.holidays_data import HolidaysData
from robyn.data.entities.hyperparameters import Hyperparameters, ChannelHyperparameters
from robyn.modeling.entities.modelrun_trials_config import TrialsConfig
from robyn.modeling.model_executor import ModelExecutor
from robyn.modeling.entities.enums import NevergradAlgorithm, Models
from robyn.modeling.feature_engineering import FeatureEngineering

## 2.1 Load Mock R data

We need to set the base path for the data directory.
Create a .env file in the same directory as your notebook and put in define the path to the data dir.
for example: ROBYN_BASE_PATH=.../Robyn/R/data

In [ ]:
# Read the simulated data and holidays data
dt_simulated_weekly = pd.read_csv(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/tutorials/resources/dt_simulated_weekly.csv"
)

dt_prophet_holidays = pd.read_csv(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/tutorials/resources/dt_prophet_holidays.csv"
)

## Setup MMM Data

We will now set up the MMM data specification which includes defining the dependent variable, independent variables, and the time window for analysis.

In [ ]:
def setup_mmm_data(dt_simulated_weekly) -> MMMData:

    mmm_data_spec = MMMData.MMMDataSpec(
        dep_var="revenue",
        dep_var_type="revenue",
        date_var="DATE",
        context_vars=["competitor_sales_B", "events"],
        paid_media_spends=["tv_S", "ooh_S", "print_S", "facebook_S", "search_S"],
        paid_media_vars=["tv_S", "ooh_S", "print_S", "facebook_I", "search_clicks_P"],
        organic_vars=["newsletter"],
        window_start="2016-01-01",
        window_end="2018-12-31",
    )

    return MMMData(data=dt_simulated_weekly, mmmdata_spec=mmm_data_spec)


mmm_data = setup_mmm_data(dt_simulated_weekly)
mmm_data.data.head()

## Feature Preprocessing

We will perform feature engineering to prepare the data for modeling. This includes transformations like adstock and other preprocessing steps.

In [ ]:
hyperparameters = Hyperparameters(
    hyperparameters={
        "facebook_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0, 0.3],
        ),
        "print_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
        "tv_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.3, 0.8],
        ),
        "search_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0, 0.3],
        ),
        "ooh_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
        "newsletter": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
    },
    adstock=AdstockType.GEOMETRIC,
    lambda_=[0, 1],
    train_size=[0.5, 0.8],
)

print("Hyperparameters setup complete.")

In [ ]:
# Create HolidaysData object
holidays_data = HolidaysData(
    dt_holidays=dt_prophet_holidays,
    prophet_vars=["trend", "season", "holiday"],
    prophet_country="DE",
    prophet_signs=["default", "default", "default"],
)
# Setup FeaturizedMMMData
feature_engineering = FeatureEngineering(mmm_data, hyperparameters, holidays_data)

In [ ]:
featurized_mmm_data = feature_engineering.perform_feature_engineering()

In [ ]:
from robyn.visualization.feature_visualization import FeaturePlotter
import matplotlib.pyplot as plt

%matplotlib inline

# Create a FeaturePlotter instance
feature_plotter = FeaturePlotter(
    mmm_data, hyperparameters, featurized_mmmdata=featurized_mmm_data
)
# Extract the list of results
results_list = featurized_mmm_data.modNLS["results"]
# Plot spend-exposure relationship for each channel in the results
for result in results_list:
    channel = result.get("channel")
    print(f"Processing channel: {channel}")  # Debugging line
    if not channel:
        print(f"Skipping invalid channel: {result}")
        continue

    try:
        fig = feature_plotter.plot_spend_exposure(channel)
        plt.show()
    except ValueError as e:
        print(f"Error plotting {channel}: {str(e)}")

In [ ]:
print(featurized_mmm_data.modNLS["plots"].keys())

for result in featurized_mmm_data.modNLS["results"]:
    print(result.get("channel"))

In [ ]:
plot_data = featurized_mmm_data.modNLS["plots"].get("facebook_I")
print(plot_data)

In [ ]:
import numpy as np


# For MMM Data
def debug_mmm_data(mmm_data):
    print("=== MMM Data Debug ===")
    print("\nMMM Data Specification:")
    print(f"Dependent Variable: {mmm_data.mmmdata_spec.dep_var}")
    print(f"Paid Media Spends: {mmm_data.mmmdata_spec.paid_media_spends}")
    print(f"Paid Media Vars: {mmm_data.mmmdata_spec.paid_media_vars}")
    print(f"Organic Vars: {mmm_data.mmmdata_spec.organic_vars}")


# For Hyperparameters
def debug_hyperparameters(hyperparameters):
    print("=== Hyperparameters Debug ===")
    print("\nHyperparameters structure:")
    for channel, params in hyperparameters.hyperparameters.items():
        print(f"\nChannel: {channel}")
        print("Thetas:", getattr(params, "thetas", None))
        print("Alphas:", getattr(params, "alphas", None))
        print("Gammas:", getattr(params, "gammas", None))
        print("Shapes:", getattr(params, "shapes", None))
        print("Scales:", getattr(params, "scales", None))

    print("\nLambda:", hyperparameters.lambda_)
    print("Train size:", hyperparameters.train_size)


# For Featurized MMM Data
def debug_featurized_data(featurized_mmm_data):
    print("=== Featurized MMM Data Debug ===")
    print("\nFeaturized data shape:", featurized_mmm_data.dt_mod.shape)
    print("\nFeaturized data columns:", featurized_mmm_data.dt_mod.columns.tolist())

    # Basic statistics of the featurized data
    print("\nFeaturized data statistics:")
    print(featurized_mmm_data.dt_mod.describe())

    # Check for any transformations that occurred
    print("\nData types:")
    print(featurized_mmm_data.dt_mod.dtypes)

    # Check for any missing or infinite values
    print("\nMissing values count:")
    print(featurized_mmm_data.dt_mod.isnull().sum())
    print("\nInfinite values count:")
    print(np.isinf(featurized_mmm_data.dt_mod.select_dtypes(include=np.number)).sum())


# Combined debug function
def debug_all_inputs(mmm_data, hyperparameters, featurized_mmm_data):
    debug_mmm_data(mmm_data)
    print("\n" + "=" * 50 + "\n")
    debug_hyperparameters(hyperparameters)
    print("\n" + "=" * 50 + "\n")
    debug_featurized_data(featurized_mmm_data)


# Usage:
debug_all_inputs(mmm_data, hyperparameters, featurized_mmm_data)

In [ ]:
# Setup ModelExecutor
model_executor = ModelExecutor(
    mmmdata=mmm_data,
    holidays_data=holidays_data,
    hyperparameters=hyperparameters,
    calibration_input=None,  # Add calibration input if available
    featurized_mmm_data=featurized_mmm_data,
)

# Setup TrialsConfig
trials_config = TrialsConfig(
    iterations=2000, trials=5
)  # Set to the number of cores you want to use

print(
    f">>> Starting {trials_config.trials} trials with {trials_config.iterations} iterations each using {NevergradAlgorithm.TWO_POINTS_DE.value} nevergrad algorithm on x cores..."
)

# Run the model

output_models = model_executor.model_run(
    trials_config=trials_config,
    ts_validation=True,  # changed from True to False -> deacitvate
    add_penalty_factor=False,
    rssd_zero_penalty=True,
    cores=16,
    nevergrad_algo=NevergradAlgorithm.TWO_POINTS_DE,
    intercept=True,
    intercept_sign="non_negative",
    model_name=Models.RIDGE,
)
print("Model training complete.")

In [ ]:
from robyn.tutorials.utils.data_mapper import load_data_from_json, import_input_collect, import_output_models
%load_ext autoreload
%autoreload 2

# Load data from JSON exported from R
raw_input_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_Pareto_50_iterations_1_trial_InputCollect.json"
)
raw_output_models = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/test_Pareto_50_iterations_1_trial_OutputModels.json"
)

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_models = import_output_models(raw_output_models)

# Extract individual components
r_mmm_data = r_input_collect["mmm_data"]
r_featurized_mmm_data = r_input_collect["featurized_mmm_data"]
r_holidays_data = r_input_collect["holidays_data"]
r_hyperparameters = r_input_collect["hyperparameters"]

In [ ]:
# new anytree

from anytree import Node, RenderTree
from dataclasses import is_dataclass, asdict
import pandas as pd


def build_tree(data, parent_key="", limit_trials=True):
    """
    Recursively build a tree structure from a dictionary, list, or dataclass.

    Args:
        data: The data structure (dict, list, or dataclass) to traverse.
        parent_key: The base key path for nested keys.
        limit_trials: Whether to limit the output to the first trial.

    Returns:
        A tree node representing the structure of the data.
    """
    if is_dataclass(data):
        data = asdict(data)  # Convert dataclass to dictionary

    if isinstance(data, dict):
        node = Node(f"{parent_key} (Dict)")
        for key, value in data.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            child_node = build_tree(value, full_key, limit_trials)
            child_node.parent = node
        return node
    elif isinstance(data, list):
        node = Node(f"{parent_key} (List)")
        for index, item in enumerate(data):
            if limit_trials and parent_key == "trials" and index > 0:
                break
            full_key = f"{parent_key}[{index}]"
            child_node = build_tree(item, full_key, limit_trials)
            child_node.parent = node
        return node
    elif isinstance(data, pd.DataFrame):
        node = Node(f"{parent_key} (DataFrame)")
        for column in data.columns:
            dtype = data[column].dtype
            column_node = Node(f"{parent_key}.{column} (dtype: {dtype})")
            column_node.parent = node
        return node
    else:
        dtype = type(data).__name__
        return Node(f"{parent_key} (dtype: {dtype})")


# Assuming output_models and r_output_models are instances of ModelOutputs
python_tree = build_tree(output_models)
r_tree = build_tree(r_output_models)

# Visualize the trees
print("Python ModelOutputs Structure:")
for pre, fill, node in RenderTree(python_tree):
    print(f"{pre}{node.name}")

print("\nR ModelOutputs Structure:")
for pre, fill, node in RenderTree(r_tree):
    print(f"{pre}{node.name}")

In [ ]:
# from anytree import Node, RenderTree
# from anytree.exporter import DotExporter
# from dataclasses import is_dataclass, asdict
# import pandas as pd


# def build_tree(data, parent_key="", limit_trials=True):
#     """
#     Recursively build a tree structure from a dictionary, list, or dataclass.

#     Args:
#         data: The data structure (dict, list, or dataclass) to traverse.
#         parent_key: The base key path for nested keys.
#         limit_trials: Whether to limit the output to the first trial.

#     Returns:
#         A tree node representing the structure of the data.
#     """
#     if is_dataclass(data):
#         data = asdict(data)  # Convert dataclass to dictionary

#     if isinstance(data, dict):
#         node = Node(parent_key)
#         for key, value in data.items():
#             full_key = f"{parent_key}.{key}" if parent_key else key
#             child_node = build_tree(value, full_key, limit_trials)
#             child_node.parent = node
#         return node
#     elif isinstance(data, list):
#         node = Node(parent_key)
#         for index, item in enumerate(data):
#             if limit_trials and parent_key == "trials" and index > 0:
#                 break
#             full_key = f"{parent_key}[{index}]"
#             child_node = build_tree(item, full_key, limit_trials)
#             child_node.parent = node
#         return node
#     elif isinstance(data, pd.DataFrame):
#         node = Node(f"{parent_key} (DataFrame: {data.shape})")
#         for column in data.columns:
#             column_node = Node(f"{parent_key}.{column}")
#             column_node.parent = node
#         return node
#     else:
#         return Node(parent_key)


# # Assuming output_models and r_output_models are instances of ModelOutputs
# python_tree = build_tree(output_models)
# r_tree = build_tree(r_output_models)

# # Visualize the trees
# print("Python ModelOutputs Structure:")
# for pre, fill, node in RenderTree(python_tree):
#     print(f"{pre}{node.name}")

# print("\nR ModelOutputs Structure:")
# for pre, fill, node in RenderTree(r_tree):
#     print(f"{pre}{node.name}")

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Any


def compare_model_values(py_output, r_output):
    """Compare key values between Python and R model outputs"""

    print("1. Basic Model Configuration Comparison:")
    basic_attrs = [
        "train_timestamp",
        "cores",
        "iterations",
        "intercept",
        "intercept_sign",
        "nevergrad_algo",
        "ts_validation",
        "add_penalty_factor",
    ]

    # Add debug prints
    print("\nDebugging attribute types:")
    for attr in basic_attrs:
        py_val = getattr(py_output, attr, None)
        r_val = getattr(r_output, attr, None)
        print(f"{attr:20s} - Python type: {type(py_val)} | R type: {type(r_val)}")
        print(f"{attr:20s} - Python value: {py_val} | R value: {r_val}")
        print("-" * 50)

    print("\n2. Trial Results Comparison (Descriptive Statistics):")
    if py_output.trials and r_output.trials:
        metrics = [
            "nrmse",
            "decomp_rssd",
            "mape",
            "rsq_train",
            "rsq_val",
            "rsq_test",
            "lambda_",
            "lambda_hp",
            "lambda_max",
            "lambda_min_ratio",
        ]
        # Convert trial results to DataFrames
        py_trials_df = pd.DataFrame(
            [
                {metric: getattr(trial, metric, np.nan) for metric in metrics}
                for trial in py_output.trials
            ]
        )

        # Aggregate R trial metrics
        r_trials_df = pd.DataFrame(
            [
                {
                    metric: getattr(trial, metric, pd.Series([np.nan])).mean()
                    for metric in metrics
                }
                for trial in r_output.trials
            ]
        )
        # Ensure R trial data is numeric
        r_trials_df = r_trials_df.apply(pd.to_numeric, errors="coerce")
        # Calculate descriptive statistics
        py_desc = py_trials_df.describe()
        r_desc = r_trials_df.describe()
        # Print descriptive statistics
        print("\nPython Trial Descriptive Statistics:")
        print(py_desc)
        print("\nR Trial Descriptive Statistics:")
        print(r_desc)
        # Calculate and print differences
        diff_desc = py_desc - r_desc
        print("\nDifference in Descriptive Statistics:")
        print(diff_desc)

    print("\n3. Hyperparameter Comparison:")
    if hasattr(py_output, "hyper_updated") and hasattr(r_output, "hyper_updated"):
        py_hyper = py_output.hyper_updated
        r_hyper = r_output.hyper_updated

        # Find all unique keys
        all_keys = set(py_hyper.keys()) | set(r_hyper.keys())

        print("\nHyperparameter Values:")
        print(f"{'Parameter':30s} {'Python':>15s} {'R':>15s} {'Diff':>15s}")
        print("-" * 75)

        for key in sorted(all_keys):
            py_val = py_hyper.get(key, "N/A")
            r_val = r_hyper.get(key, "N/A")

            if isinstance(py_val, (int, float)) and isinstance(r_val, (int, float)):
                diff = abs(py_val - r_val)
                print(f"{key:30s} {py_val:15.6f} {r_val:15.6f} {diff:15.6f}")
            else:
                print(f"{key:30s} {str(py_val):15s} {str(r_val):15s} {'N/A':>15s}")

    print("\n4. Data Shape Comparison:")
    data_attrs = ["all_result_hyp_param", "all_x_decomp_agg", "all_decomp_spend_dist"]

    for attr in data_attrs:
        py_shape = getattr(py_output, attr).shape if hasattr(py_output, attr) else None
        r_shape = getattr(r_output, attr).shape if hasattr(r_output, attr) else None
        print(f"{attr:20s} - Python shape: {py_shape} | R shape: {r_shape}")

    print("\n5. Total Effect and Effect Share Comparison:")
    if py_output.trials and r_output.trials:
        for i, (py_trial, r_trial) in enumerate(zip(py_output.trials, r_output.trials)):
            print(f"\nTrial {i+1}:")
            # Extract decomp_spend_dist DataFrames
            py_decomp_spend_dist = getattr(
                py_trial, "decomp_spend_dist", pd.DataFrame()
            )
            r_decomp_spend_dist = getattr(r_trial, "decomp_spend_dist", pd.DataFrame())
            # Check if the DataFrames are not empty
            if not py_decomp_spend_dist.empty and not r_decomp_spend_dist.empty:
                # Compare total effect
                py_total_effect = py_decomp_spend_dist.get(
                    "xDecompAgg", pd.Series([np.nan])
                ).sum()
                r_total_effect = r_decomp_spend_dist.get(
                    "xDecompAgg", pd.Series([np.nan])
                ).sum()
                total_effect_diff = abs(py_total_effect - r_total_effect)
                print(
                    f"Total Effect - Python: {py_total_effect:.6f}, R: {r_total_effect:.6f}, Diff: {total_effect_diff:.6f}"
                )
                # Compare effect share
                py_effect_share = py_decomp_spend_dist.get(
                    "effect_share", pd.Series([np.nan])
                ).sum()
                r_effect_share = r_decomp_spend_dist.get(
                    "effect_share", pd.Series([np.nan])
                ).sum()
                effect_share_diff = abs(py_effect_share - r_effect_share)
                print(
                    f"Effect Share - Python: {py_effect_share:.6f}, R: {r_effect_share:.6f}, Diff: {effect_share_diff:.6f}"
                )
            else:
                print(
                    "Decomposition spend distribution data is missing for this trial."
                )


# Run the comparison
print("Starting detailed value comparison...\n")
compare_model_values(output_models, r_output_models)

In [ ]:
# Assuming r_output_models is an instance of ModelOutputs
convergence_data = r_output_models.convergence

# Accessing attributes of ConvergenceData
moo_distrb_plot = convergence_data.moo_distrb_plot
moo_cloud_plot = convergence_data.moo_cloud_plot
errors = convergence_data.errors
conv_msg = convergence_data.conv_msg

# Example: Print the convergence messages
print("Convergence Messages:", conv_msg)

In [ ]:
from PIL import Image
import io
import binascii
from typing import Optional

# Assuming convergence_data is an instance of ConvergenceData
# and contains hexadecimal strings of image data


def display_image(data: Optional[str]):
    if data:
        # If data is a list, join it into a single string
        if isinstance(data, list):
            data = "".join(data)

        # Convert hexadecimal string to binary data
        binary_data = binascii.unhexlify(data)

        # Open the image using PIL
        image = Image.open(io.BytesIO(binary_data))

        # Display the image
        image.show()
    else:
        print("No image data available.")


# print("moo distrb plot:", type(moo_distrb_plot))
# print("first 10 elements in moo distrb plot:", moo_distrb_plot[0:10])
# print("moo cloud plot:", type(moo_cloud_plot))
# Assuming convergence_data is an instance of ConvergenceData
# Display the moo_distrb_plot
# display_image(convergence_data.moo_distrb_plot)
# # Display the moo_cloud_plot
# display_image(convergence_data.moo_cloud_plot)

In [ ]:
# # Function to print model output summary
# def print_model_output_summary(model_name, model):
#     print(f"\n{model_name} Model Output Summary:")
#     print(f"Number of trials: {len(model.trials)}")
#     print(
#         f"Average models per trial: {len(model.all_result_hyp_param) / len(model.trials)}"
#     )
#     print(f"Total unique models: {len(model.all_result_hyp_param['sol_id'].unique())}")

#     print("\nMetrics Distribution:")
#     metrics_df = model.all_result_hyp_param[["nrmse", "decomp.rssd", "mape"]]
#     print(metrics_df.describe())

#     # Additional validation to debug model output
#     print("\nColumns in result_hyp_param:")
#     print(model.all_result_hyp_param.columns.tolist())

#     print("\nSample rows of metrics:")
#     print(model.all_result_hyp_param[["sol_id", "nrmse", "decomp.rssd", "mape"]].head())

#     # Show shape of result dataframes
#     print("\nDataFrame Shapes:")
#     print(f"result_hyp_param: {model.all_result_hyp_param.shape}")
#     print(f"x_decomp_agg: {model.all_x_decomp_agg.shape}")
#     print(f"decomp_spend_dist: {model.all_decomp_spend_dist.shape}")


# # Print summaries for both output_models and r_output_models
# print_model_output_summary("Python Output Models", output_models)
# print_model_output_summary("R Output Models", r_output_models)

In [ ]:
# def print_data_structure(data):
#     print("Columns:", data.columns.tolist())
#     print("\nFirst row:", data.iloc[0].to_dict())
#     print("\nShape:", data.shape)


# # Assuming you want to print the structure for the first trial
# first_trial_r = r_output_models.trials[0].decomp_spend_dist
# first_trial_python = output_models.trials[0].decomp_spend_dist

# print("R exported data structure:")
# print_data_structure(first_trial_r)

# # With Python calculated data
# print("\nPython calculated data structure:")
# print_data_structure(first_trial_python)